# Feb 28, 2024: Yale's parcellations
modified: [this ipynb](../dec23/00-desc-coarse-parcels.ipynb)

In [1]:
import csv
import os
import numpy as np 
import pandas as pd 
import ants
import json
from tqdm import tqdm 

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

In [3]:
BASE_path = f'{os.environ["HOME"]}/mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
os.system(f'mkdir -p {PARCELS_path}')

# 172 rois
parcels_172 = (
    f'{BASE_path}/n162_parcellations'
    f'/172_rois_N162_merge_medium_fine_224_0.2_RAS.nii'
)
parcels_172 = ants.image_read(parcels_172)

with open(f'{BASE_path}/n162_parcellations/172_rois.json') as f:
    roi_names_172 = json.load(f)['labels']['data'][0]['labels']
    roi_names_172 = pd.DataFrame({
        'roi': [int(k) for k in list(roi_names_172.keys())],
        'label': list(roi_names_172.values())
    })

# 56 rois
parcels_56 = (
    f'{BASE_path}/n162_parcellations'
    f'/56_rois_new_N162_coarse_medium_224_noskip_0.2_RAS.nii'
)
parcels_56 = ants.image_read(parcels_56)

roi_names_56 = pd.read_csv(f'{BASE_path}/n162_parcellations/56_rois.csv', names=['roi', 'label'])

In [4]:
n162_ras_file = f'/home/govindas/mouse_dataset/gabe_symmetric_N162/Symmetric_N162_0.20_RAS.nii.gz'
n162_ras = ants.image_read(n162_ras_file)

In [5]:
def orient_parcels(n162_ras, parcels):
    n = parcels.numpy()
    n = n.transpose(0, 2, 1)
    parcels_xf = n162_ras.new_image_like(n)
    return parcels_xf

In [6]:
parcels_172_ras = orient_parcels(n162_ras, parcels_172)
parcels_172_ras.to_file(f'{BASE_path}/n162_parcellations/172_parcels_RAS.nii.gz')

parcels_56_ras = orient_parcels(n162_ras, parcels_56)
parcels_56_ras.to_file(f'{BASE_path}/n162_parcellations/56_parcels_RAS.nii.gz')

In [7]:
# common brain mask (across subs)
BASE_path = f'{os.environ["HOME"]}/mouse_dataset'
all_files_path = f'{BASE_path}/voxel/all_file_collections'
all_files = os.listdir(all_files_path)

# cmask : common brain mask
for idx, files in tqdm(enumerate(all_files[:])):
    if idx == 0:
        with open(f'{all_files_path}/{files}', 'r') as f:
            cmask_img = ants.image_read(f.readlines()[1][:-1])
        cmask = cmask_img.numpy()
    else:
        with open(f'{all_files_path}/{files}', 'r') as f:
            cmask *= ants.image_read(f.readlines()[1][:-1]).numpy()
cmask_img = cmask_img.new_image_like(cmask)
cmask_img.to_filename(
    f'{BASE_path}/voxel/common_brain_mask.nii.gz'
)

116it [00:00, 361.92it/s]


In [8]:
def resample(args, target, img):
    img_rs = ants.resample_image_to_target(
        image=img,
        target=target,
        interp_type='genericLabel',
    )
    img_rs = img_rs.new_image_like(
        data=img_rs.numpy() * (target.numpy() > 0)
    )
    print(img_rs.numpy().shape)
    return img_rs

In [9]:
# isocortex + subcortex rois
rois_172 = np.arange(1, 173, 1)
iso_rois_172 = np.array(
    list(range(1, 44)) +
    list(range(88, 130))
)
sub_rois_172 = np.array(
    list(range(47, 87)) +
    list(range(133, 173))
)
# print(iso_rois_172, sub_rois_172)

rois_56 = np.arange(1, 57, 1)
iso_rois_56 = np.array(
    list(range(1, 18)) +
    list(range(29, 46))
)
sub_rois_56 = np.array(
    list(range(19, 29)) + 
    list(range(47, 57))
)
# print(iso_rois_56, sub_rois_56)

In [10]:
def create_sub_parcels(parcels, rois, name):
    parcels_new = np.zeros_like(parcels.numpy())
    for roi in rois:
        parcels_new += roi * (parcels.numpy() == roi)
    parcels_new_img = parcels.new_image_like(
        data=parcels_new,
    )
    return parcels_new_img

parcels_172_iso_img = create_sub_parcels(parcels_172_ras, iso_rois_172, '172_parcels_RAS_iso')
parcels_172_sub_img = create_sub_parcels(parcels_172_ras, sub_rois_172, '172_parcels_RAS_sub')

parcels_56_iso_img = create_sub_parcels(parcels_56_ras, iso_rois_56, '56_parcels_RAS_iso')
parcels_56_sub_img = create_sub_parcels(parcels_56_ras, sub_rois_56, '56_parcels_RAS_sub')

In [11]:
def save_parcels(args, parcels, base_parcels, cmask_img):
    parcels_name = (
        f'type-{args.type}'
        f'_size-{args.roi_size}'
        f'_symm-{args.maintain_symmetry}'
        f'_braindiv-{args.brain_div}'
        f'_nrois-{args.num_rois}'
        f'_desc-parcels.nii.gz'
    )
    file = f'{PARCELS_path}/{parcels_name}'
    PARCELS_img = base_parcels.new_image_like(parcels.astype(np.uint32))
    PARCELS_cm_img = resample(args, target=cmask_img, img=PARCELS_img)
    PARCELS_cm_img.to_filename(file)
    return file

def roi_labels(args, mask_file):
    # file with roi labels
    labels_name = (
        f'type-{args.type}'
        f'_size-{args.roi_size}'
        f'_symm-{args.maintain_symmetry}'
        f'_braindiv-{args.brain_div}'
        f'_nrois-{args.num_rois}'
        f'_desc-labels.txt'
    )
    cmd = (
        f'3dROIstats -overwrite '
        f'-quiet '
        f'-mask {mask_file} '
        f'{mask_file} > {PARCELS_path}/{labels_name}'
    )
    os.system(cmd)
    return None

def roi_names(args, roi_names_df, labels):
    roi_names_df = roi_names_df[roi_names_df['roi'].isin(labels)]
    roi_names = list(zip(
        roi_names_df['roi'], roi_names_df['label']
    ))
    
    file_name = (
        f'type-{args.type}'
        f'_size-{args.roi_size}'
        f'_symm-{args.maintain_symmetry}'
        f'_braindiv-{args.brain_div}'
        f'_nrois-{args.num_rois}'
        f'_desc-names.csv'
    )

    with open(f'{PARCELS_path}/{file_name}', 'w', newline='') as f:
        wr = csv.writer(f, delimiter='\t')
        wr.writerows(roi_names)
    
    return None

In [12]:
args.type = 'allen'
args.roi_size = 'x'
args.maintain_symmetry = 'True'

# 172 rois
parcels_img = parcels_172_ras
args.brain_div = 'whl'
args.num_rois = len(np.unique(parcels_img.numpy())[1:])
file = save_parcels(args, parcels_img.numpy(), parcels_img, cmask_img)
roi_labels(args, file)
roi_names(args, roi_names_172, rois_172)

parcels_img = parcels_172_iso_img
args.brain_div = 'ctx'
args.num_rois = len(np.unique(parcels_img.numpy())[1:])
file = save_parcels(args, parcels_img.numpy(), parcels_img, cmask_img)
roi_labels(args, file)
roi_names(args, roi_names_172, iso_rois_172)

parcels_img = parcels_172_sub_img
args.brain_div = 'sub'
args.num_rois = len(np.unique(parcels_img.numpy())[1:])
file = save_parcels(args, parcels_img.numpy(), parcels_img, cmask_img)
roi_labels(args, file)
roi_names(args, roi_names_172, sub_rois_172)

# 56 rois
parcels_img = parcels_56_ras
args.brain_div = 'whl'
args.num_rois = len(np.unique(parcels_img.numpy())[1:])
file = save_parcels(args, parcels_img.numpy(), parcels_img, cmask_img)
roi_labels(args, file)
roi_names(args, roi_names_56, rois_56)

parcels_img = parcels_56_iso_img
args.brain_div = 'ctx'
args.num_rois = len(np.unique(parcels_img.numpy())[1:])
file = save_parcels(args, parcels_img.numpy(), parcels_img, cmask_img)
roi_labels(args, file)
roi_names(args, roi_names_56, iso_rois_56)

parcels_img = parcels_56_sub_img
args.brain_div = 'sub'
args.num_rois = len(np.unique(parcels_img.numpy())[1:])
file = save_parcels(args, parcels_img.numpy(), parcels_img, cmask_img)
roi_labels(args, file)
roi_names(args, roi_names_56, sub_rois_56)

(58, 79, 45)
(58, 79, 45)
(58, 79, 45)
(58, 79, 45)
(58, 79, 45)
(58, 79, 45)
